Inicio do WebScraping

O bloco de código abaixo usa Selenium para abrir o site IMDb para captura lista dos 250 melhores filmes do site e depois armazena como texto na variável lista_filmes. 

In [60]:
from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import time
import re

options = webdriver.ChromeOptions()
service = Service(ChromeDriverManager().install())
drv = webdriver.Chrome(service=service, options=options)

url = 'https://www.imdb.com/pt/'
drv.get(url)

time.sleep(5)

drv.find_element(By.ID, 'imdbHeader-navDrawerOpen').click()

time.sleep(5)

drv.find_elements(By.TAG_NAME, 'a')[2].click()

time.sleep(5)

lista_filmes = drv.find_element(By.XPATH, '/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul').text

 Neste bloco acontece a extração dos dados de cada filme, como posição, título, ano, duração, classificação, avaliação e quantidade de avaliações, a partir do texto lista_filmes. 
 
Ele armazena esses dados em um dicionário para cada filme e os adiciona a uma lista. No final, os dados são convertidos em um DataFrame do pandas para análise.

In [70]:
filme_padrao = re.compile(r'(\d+)\.\s([^\n]+)\n(\d{4})\n([^\n]+)\n([^\n]+)\n([^\n]+)\n([^\n]+)')
comeco = filme_padrao.findall(lista_filmes)

movie_data = []

for ponto_inicial in comeco:
    movie = {
        "Posição": ponto_inicial[0],
        "Título": ponto_inicial[1],
        "Ano": ponto_inicial[2],
        "Duração": ponto_inicial[3],
        "Classificação": ponto_inicial[4],
        "Avaliação": ponto_inicial[5],
        "QTD_Avaliação": ponto_inicial[6]
    }
    movie_data.append(movie)

df = pd.DataFrame(movie_data)


Nessa parte final, O código é limpo, convertendo a coluna de 'Duração' para um formato de tempo, separando e limpando a coluna 'QTD_Avaliação' em duas partes. Em seguida, converte 'Total_Avaliação' para valores numéricos e calcula uma nova coluna 'Avaliação_total' multiplicando pelo valor da unidade ('mi' ou 'mil'). Por fim, remove colunas desnecessárias.

In [71]:
df['Duração'] = pd.to_timedelta(df['Duração'].str.replace('h', ' hours').str.replace('m', ' minutes'))

df['Duração'] = df['Duração'].astype(str).str.replace("0 days ", "")

df['QTD_Avaliação'] = df['QTD_Avaliação'].str.replace('(','').str.replace(')','')

df[['Total_Avaliação', 'UND']] = df['QTD_Avaliação'].str.strip().str.split(' ', n=1, expand=True)

df.drop(columns=['QTD_Avaliação'], inplace=True)

df['Total_Avaliação'] = df['Total_Avaliação'].str.replace(',', '.')
df['Total_Avaliação'] = pd.to_numeric(df['Total_Avaliação'], errors='coerce')

df['Avaliação_total'] = df['Total_Avaliação']*df['UND'].map({'mi':1000000,'mil':1000})
df.drop(columns=['Total_Avaliação'], inplace=True)
df.drop(columns=['UND'], inplace=True)

df.head()

,Posição,Título,Ano,Duração,Classificação,Avaliação,Avaliação_total
0,1,Um Sonho de Liberdade,1994,02:22:00,16,"9,3",3000000.0
1,2,O Poderoso Chefão,1972,02:55:00,14,"9,2",2100000.0
2,3,Batman: O Cavaleiro das Trevas,2008,02:32:00,12,"9,0",3000000.0
3,4,O Poderoso Chefão: Parte II,1974,03:22:00,14,"9,0",1400000.0
4,5,12 Homens e uma Sentença,1957,01:36:00,Livre,"9,0",911000.0
